<a href="https://colab.research.google.com/github/KENG85/colab_notebooks/blob/main/NBA_sentiment_scrapenclean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mount your google drive to create a secure connection
def mount_google_drive(force_remount=True):
    from google.colab import drive
    print('drive_filepath="drive/My Drive/"')
    return drive.mount('/content/drive', force_remount=force_remount)

In [ ]:
#run the connection
mount_google_drive()

drive_filepath="drive/My Drive/"
Mounted at /content/drive


In [ ]:
# Dependencies
import pandas as pd
import json
#import gmaps
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import google
from bs4 import BeautifulSoup as bs, NavigableString as ns
from pprint import pprint

In [ ]:
#archive page
transcript_url = 'http://www.asapsports.com/show_event.php?category=11&date=2021-7-10&title=NBA+FINALS%3A+BUCKS+VS.+SUNS'

# Use requests to retrieve data from a given URL
response = requests.get(transcript_url)

# Parse the whole HTML page using BeautifulSoup
july5_soup = bs(response.text, 'html.parser')

# Title of the parsed page
title = july5_soup.title.string
print(title)

ASAP Sports Transcripts - Basketball - 2021 - NBA FINALS: BUCKS VS. SUNS - July 10


In [ ]:
links = []
for link in july5_soup.findAll('a', attrs={'href': re.compile("^http://")}):
        links.append(link.get('href'))
links

['http://www.asapsports.com',
 'http://www.asapsports.com',
 'http://www.facebook.com/pages/ASAP-Sports/122368829031',
 'http://twitter.com/asapsports',
 'http://www.asaptext.com',
 'http://www.asaptext.com',
 'http://www.asapsports.com/show_interview.php?id=167289',
 'http://www.asapsports.com/show_interview.php?id=167286',
 'http://www.asapsports.com/show_interview.php?id=167294',
 'http://www.asapsports.com/show_interview.php?id=167287',
 'http://www.asapsports.com/show_interview.php?id=167290',
 'http://www.asapsports.com/show_interview.php?id=167288',
 'http://www.asapsports.com/show_interview.php?id=167291',
 'http://www.asapsports.com/show_interview.php?id=167285',
 'http://www.asapsports.com/show_interview.php?id=167295',
 'http://www.asapsports.com/show_interview.php?id=167292',
 'http://www.asapsports.com/show_interview.php?id=167293',
 'http://www.asapsports.com/show_interview.php?id=167296']

In [ ]:
interview = "http://www.asapsports.com/show_interview.php?id="
ids = [167062,167069,167071,167066,167059,167060,167070,167067,167061,167068,167064,167065,167072,167058,167063,167057]
titles = []
text = []
docs = []
for id in ids:
    url = f'{interview}+{id}'
    visit = requests.get(url, timeout=30)
    html = bs(visit.text, 'html.parser')
    title = html.title.string
    #doc = html.get_text()
    for para in html.find_all("p", text=True):
      doc = para.get_text()
    #answers = html.find_all('p', text=True)
    titles.append(title)
    #text.append(answers)
    docs.append(doc)


In [ ]:
docs = pd.DataFrame(docs)
#docs = docs[0].rename("Text")
docs

,0
0,MONTY WILLIAMS: I think the guys feel really...
1,"So, a lot of things have to change and have to..."
2,JEFF TEAGUE: It's just a huge opportunity. ...
3,"So obviously, each series has been different, ..."
4,All of it comes together as one. He makes sur...
5,"CHRIS PAUL: Yeah, a big piece of their team...."
6,KHRIS MIDDLETON: I think everything we have ...
7,"BROOK LOPEZ: Yeah, it's definitely been biza..."
8,"CAMERON JOHNSON: Number one, they're good pl..."
9,JRUE HOLIDAY: I think one of the keys is jus...


In [ ]:
titles = pd.DataFrame(titles)
titles = titles[0].str.replace('ASAP Sports Transcripts - Basketball - 2021 - NBA FINALS: BUCKS VS. SUNS - ','')
titles = pd.DataFrame(titles)
titles

,0
0,July 5 - Monty Williams
1,July 5 - P.J. Tucker
2,July 5 - Jeff Teague
3,July 5 - Bobby Portis
4,July 5 - Cameron Payne
5,July 5 - Chris Paul
6,July 5 - Khris Middleton
7,July 5 - Brook Lopez
8,July 5 - Cameron Johnson
9,July 5 - Jrue Holiday


In [ ]:
docs = docs[0].astype(str)
docs.dtypes

dtype('O')

In [ ]:
import re
import string

#Create a function to clean the text
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    #text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<p>', '', text)
    text = re.sub('</p>', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
clean_doc = pd.DataFrame(docs.apply(round1))
clean_doc

,0
0,MONTY WILLIAMS I think the guys feel really ...
1,So a lot of things have to change and have to ...
2,JEFF TEAGUE Its just a huge opportunity Its...
3,So obviously each series has been different bu...
4,All of it comes together as one He makes sure...
5,CHRIS PAUL Yeah a big piece of their team T...
6,KHRIS MIDDLETON I think everything we have b...
7,BROOK LOPEZ Yeah its definitely been bizarre...
8,CAMERON JOHNSON Number one theyre good playe...
9,JRUE HOLIDAY I think one of the keys is just...


In [ ]:
# Create a function to apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
clean_doc = pd.DataFrame(clean_doc[0].apply(round2))
clean_doc

,0
0,MONTY WILLIAMS I think the guys feel really ...
1,So a lot of things have to change and have to ...
2,JEFF TEAGUE Its just a huge opportunity Its...
3,So obviously each series has been different bu...
4,All of it comes together as one He makes sure...
5,CHRIS PAUL Yeah a big piece of their team T...
6,KHRIS MIDDLETON I think everything we have b...
7,BROOK LOPEZ Yeah its definitely been bizarre...
8,CAMERON JOHNSON Number one theyre good playe...
9,JRUE HOLIDAY I think one of the keys is just...


In [ ]:
clean_doc["id"] = titles
clean_doc

,0,id
0,MONTY WILLIAMS I think the guys feel really ...,July 5 - Monty Williams
1,So a lot of things have to change and have to ...,July 5 - P.J. Tucker
2,JEFF TEAGUE Its just a huge opportunity Its...,July 5 - Jeff Teague
3,So obviously each series has been different bu...,July 5 - Bobby Portis
4,All of it comes together as one He makes sure...,July 5 - Cameron Payne
5,CHRIS PAUL Yeah a big piece of their team T...,July 5 - Chris Paul
6,KHRIS MIDDLETON I think everything we have b...,July 5 - Khris Middleton
7,BROOK LOPEZ Yeah its definitely been bizarre...,July 5 - Brook Lopez
8,CAMERON JOHNSON Number one theyre good playe...,July 5 - Cameron Johnson
9,JRUE HOLIDAY I think one of the keys is just...,July 5 - Jrue Holiday


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#DTM for question 1
#cv = CountVectorizer(stop_words='english')
#clean_cv = cv.fit_transform(clean_doc[0])
#clean_dtm = pd.DataFrame(clean_cv.toarray(), columns=cv.get_feature_names())
#clean_dtm.index = titles.index
#clean_dtm[["id"]]= titles[0]
#clean_dtm

In [ ]:
#pickle it for later use
clean_dtm.to_pickle("drive/My Drive/Colab Notebooks/NBA Finals Sentiment Analysis/clean_dtm.pkl")
clean_doc.to_csv("drive/My Drive/Colab Notebooks/NBA Finals Sentiment Analysis/clean_doc.csv")
#!cp redo_dtm1.csv "drive/My Drive/Colab Notebooks/"
#!cp redo_dtm1.pkl "drive/My Drive/Colab Notebooks/"